In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

# PART 1 - Song classification and recommendation

In [3]:
df_rec = pd.read_csv("datasets/dataset_part_1/recommendation_spotify.csv")

In [13]:
df_rec.head(3)

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
0,0.991,['Mamie Smith'],0.598,168333,0.224,0,0cS0A1fUEUd1EW3FcF8AEI,0.000522,5,0.3790,-12.628,0,Keep A Song In Your Soul,12,1920,0.0936,149.976,0.634,1920
1,0.643,"[""Screamin' Jay Hawkins""]",0.852,150200,0.517,0,0hbkKFIJm7Z05H8Zl9w30f,0.026400,5,0.0809,-7.261,0,I Put A Spell On You,7,1920-01-05,0.0534,86.889,0.950,1920
2,0.993,['Mamie Smith'],0.647,163827,0.186,0,11m7laMUgmOKqI3oYzuhne,0.000018,0,0.5190,-12.098,1,Golfing Papa,4,1920,0.1740,97.600,0.689,1920


In [ ]:
print(f"{df_rec.shape[0]} chansons disponibles.")

174389 chansons disponibles


## **Exercice 3**

Pour le choix des features, on sélectionne les variables numériques intrinsèques aux musique.  
Pas `year` car on ne veut pas prendre le risque de recommander que des chansons de la même année dans un système de recommentdation générique.  
Pas `duration_ms` car on ne veut pas grouper les chansons par leur durée, çe ne nous dit pas grand chose sur le sons.  
Pas `explicit` car on risque de découper notre dataset en deux parite étanche alors que une musique peut exister en version clean et explicit donc être très proche.

In [6]:
features = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'valence']

In [10]:
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_rec[features])

df_features = pd.DataFrame(df_scaled, columns=features)

In [14]:
df_features.head(3)

,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence
0,1.294358,0.347919,-0.948791,-0.588004,0.930106,-0.154111,-0.066549,1.089753,0.413903
1,0.378411,1.790898,0.125710,-0.510657,-0.721489,0.788862,-0.287113,-0.995485,1.608718
2,1.299622,0.626289,-1.088146,-0.589511,1.705763,-0.060991,0.374580,-0.641450,0.621861


### 1. Première approche : Similarité cosinus "Brute"

Pour répondre à la consigne de grouper les chansons par leurs caractéristiques audio, nous utilisons la **similarité cosinus**. Cette mesure calcule l'angle entre deux vecteurs (les caractéristiques normalisées de deux chansons) pour déterminer leur proximité.

1.  **Extraction** du vecteur de caractéristiques de la chanson choisie.
2.  **Calcul** de la similarité avec l'intégralité du dataset (près de 175 000 titres).
3.  **Sélection** des 10 titres ayant le score le plus proche de 1.

> **Note sur le problème des doublons :** > Dans un dataset musical comme Spotify, il est fréquent qu'un même titre apparaisse plusieurs fois (versions Remastered, Radio Edit, ou présence sur différentes compilations). Mathématiquement, ces versions sont quasiment identiques, ce qui s'avère être un défaut pour l'expérience utilisateur qui reçoit plusieurs fois la même chanson.

In [28]:
def recommend_playlist(song_name, n_recommendations=10):
    try:
        idx = df_rec[df_rec["name"].str.lower() == song_name.lower()].index[0] # On vérifie que la chanson existe
    except IndexError:
        return "Sorry, this song is not in our database."


    similarity_score = cosine_similarity([df_scaled[idx]], df_scaled)[0]

    close_indices = similarity_score.argsort()[-(n_recommendations+1):-1][::-1]

    return df_rec.iloc[close_indices][["name", "artists", "year"]]


In [29]:
#song = df_rec.iloc[0]['name']
song = "Poker Face"

print(f"Playlist recommandée pour : {song}")
recommend_playlist(song)

Playlist recommandée pour : Poker Face


,name,artists,year
16141,Poker Face,['Lady Gaga'],2020
140736,Poker Face,['Lady Gaga'],2009
17502,Poker Face,['Lady Gaga'],2008
135137,Freaky Dancin',['Cameo'],1981
173945,Bad Boy,"['Yung Bae', 'bbno$', 'Billy Marchiafava']",2019
135066,Ein Jahr (Es geht voran),['Fehlfarben'],1980
71686,Don't Stop 'Til You Get Enough,['Michael Jackson'],1995
16461,Like I Love You,['Justin Timberlake'],2002
135713,I Cry Just a Little Bit,"[""Shakin' Stevens""]",1983
139059,Aaron's Party (Come Get It),['Aaron Carter'],2000


### 2. Version améliorée : Filtrage et diversification

Afin de proposer une playlist réellement exploitable, nous avons optimisé la fonction de recommandation pour supprimer les redondances et favoriser la découverte.

**Améliorations apportées :**
* **Recherche élargie :** Au lieu de prendre les $n$ meilleures, nous analysons les $5 \times n$ chansons les plus proches. ($n$ le nombre de chansons dans la playlist)
* **Filtrage des doublons :** Nous implémentons une vérification sur les noms de titres. Si un titre a déjà été ajouté à la sélection, les versions alternatives (remasters, etc.) sont ignorées.
* **Exclusion de l'originale :** La chanson de départ est systématiquement retirée des suggestions.

**Résultat :**
Comme illustré par le test sur *"Poker Face"*, le modèle parvient désormais à sortir de l'univers strict de Lady Gaga pour proposer des "jumeaux sonores" cohérents (Michael Jackson, Justin Timberlake, Britney Spears), tout en respectant la texture audio de l'époque et du style.

In [ ]:
def recommend_playlist_clean(song_name, n_recommendations=10):
    try:
        idx = df_rec[df_rec["name"].str.lower() == song_name.lower()].index[0]
        original_name = df_rec.iloc[idx]["name"]
        
        similarity_score = cosine_similarity([df_scaled[idx]], df_scaled)[0]
        
        indices_potents = similarity_score.argsort()[-5*n_recommendations:][::-1]
        
        recommendations = []
        already_seen = {original_name.lower()} # On ignore l'originale
        
        for i in indices_potents:
            row = df_rec.iloc[i]
            name = row["name"].lower()
            
            if name not in already_seen:
                recommendations.append(row)
                already_seen.add(name)
            
            if len(recommendations) >= n_recommendations:
                break
                
        return pd.DataFrame(recommendations)[["name", "artists", "year"]]
    
    except IndexError:
        return "Sorry, this song is not in our database."


In [35]:
print(f"Playlist {song} sans doublons :")
recommend_playlist_clean(song)

Playlist Poker Face sans doublons :


,name,artists,year
135137,Freaky Dancin',['Cameo'],1981
173945,Bad Boy,"['Yung Bae', 'bbno$', 'Billy Marchiafava']",2019
135066,Ein Jahr (Es geht voran),['Fehlfarben'],1980
71686,Don't Stop 'Til You Get Enough,['Michael Jackson'],1995
16461,Like I Love You,['Justin Timberlake'],2002
135713,I Cry Just a Little Bit,"[""Shakin' Stevens""]",1983
139059,Aaron's Party (Come Get It),['Aaron Carter'],2000
17614,Circus,['Britney Spears'],2008
140320,Rock This Party - Everybody Dance Now,"['Bob Sinclar', 'Cutee B.', 'DollarMan', 'Big ...",2006
39060,Ride It,['Regard'],2019
